In [264]:
import pandas as pd
import numpy as np
import jieba
import nltk
import os
import xlrd
import chardet
import json
import re
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim import models
from gensim.corpora import Dictionary

In [256]:
data_meta = {
    "comment_id" : 0,
    "student_id" : 1,
    "create_time" : 2,
    "teacher_rating" : 3,
    "network_rating" : 4,
    "courseware_rating" : 5,
    "content_teacher" : 6,
    "class_level" : 7,
    "unit_name" : 8,
    "lesson_name" : 9,
    "class_id" : 10,
}
def read_data(in_root_path):
    #in_root_path = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback"
    ret = []
    file_list = os.listdir(in_root_path)
    for each_file in file_list:
        print(os.path.join(in_root_path,each_file))
        with open(os.path.join(in_root_path,each_file),encoding="utf-8") as in_tmp:
            for each_line in in_tmp:
                #print(each_line)
                #each_line = each_line.rstrip("\n")
                ret.append(each_line.split("\001"))
    return ret

def preprocess_data(in_root_path,out_path,stop_words):
    with open(out_path,"w") as out_handler:
        file_list = os.listdir(in_root_path)
        for each_file in file_list:
            print(os.path.join(in_root_path,each_file))
            with open(os.path.join(in_root_path,each_file),encoding="utf-8") as in_tmp:
                for each_line in in_tmp:
                    #print(each_line)
                    #each_line = each_line.rstrip("\n")
                    data_fields = each_line.split("\001")
                    content = jieba.cut(data_fields[data_meta["content_teacher"]],cut_all=False)
                    word_arr = []
                    for each_word in content:
                        if not each_word in stop_words:
                            word_arr.append(each_word)
                    out_str = " ".join(word_arr)
                    out_str = out_str.strip()
                    if out_str and len(out_str)>0:
                        out_handler.write(out_str+"\n")
                        out_handler.flush()
            #break
sentence_pat = re.compile('。|！|\!|\.|？|\?|;|；|,|，')           
def cn_cent_tokenize(text):
    sentences = sentence_pat.split(text)         # 保留分割符
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        new_sents.append(sent)
    return new_sents

def preprocess_data_v2(in_root_path,stop_words):
    good_rate_words = {}
    bad_rate_words = {}
    good_df = 0
    bad_df = 0
    #with open(out_path,"w") as out_handler:
    file_list = os.listdir(in_root_path)
    for each_file in file_list:
        print(os.path.join(in_root_path,each_file))
        with open(os.path.join(in_root_path,each_file),encoding="utf-8") as in_tmp:
            for each_line in in_tmp:
                #print(each_line)
                #each_line = each_line.rstrip("\n")
                data_fields = each_line.split("\001")
                content = jieba.cut(data_fields[data_meta["content_teacher"]],cut_all=False)
                #content = nltk.sent_tokenize(data_fields[data_meta["content_teacher"]])
                #content = cn_cent_tokenize(data_fields[data_meta["content_teacher"]])
                teacher_rate = data_fields[data_meta["teacher_rating"]]
                #word_arr = []
                if teacher_rate and len(teacher_rate.strip())>0:
                    if teacher_rate > "3":
                        good_df += 1
                    else:
                        bad_df += 1
                    word_tmp = {}
                    for each_word in content:
                        if not each_word in stop_words:
                            #word_arr.append(each_word)

                                if teacher_rate > "3":
                                    if each_word in good_rate_words:
                                        #good_rate_words[each_word]["tf"] += 1
                                        if not each_word in word_tmp :
                                            good_rate_words[each_word]["tf"] += 1
                                            good_rate_words[each_word]["df"] += 1
                                            word_tmp[each_word] = 1
                                    else:
                                        good_rate_words[each_word]={"tf":1,"df":1}
                                else:
                                    if each_word in bad_rate_words:
                                        #bad_rate_words[each_word]["tf"] += 1
                                        if not each_word in word_tmp :
                                            bad_rate_words[each_word]["tf"] += 1
                                            bad_rate_words[each_word]["df"] += 1
                                            word_tmp[each_word] = 1
                                    else:
                                        bad_rate_words[each_word]={"tf":1,"df":1}
        #break
    return good_rate_words,bad_rate_words,good_df,bad_df

def preprocess_teacher_star(in_root_path):
    tea_star_data = {}
    file_list = os.listdir(in_root_path)
    for each_file in file_list:
        print(os.path.join(in_root_path,each_file))
        with open(os.path.join(in_root_path,each_file),encoding="utf-8") as in_tmp:
            for each_line in in_tmp:
                data_fields = each_line.split("\001")
                teacher_id = data_fields[data_meta["content_teacher"]]
                teacher_star = data_fields[data_meta["teacher_rating"]]

def read_tsinghua():
    p_set = set()
    n_set = set()
    in_path = ""
    
    with open(os.path.join(in_path,"tsinghua.negative.gb.txt")) as in_handler :
        for each_word in in_handler:
            each_word = each_word.strip()
            n_set.add(each_word)
    return p_set,n_set

def read_file(in_path,out_set,p_encoding="utf-8"):
    print(in_path)
    with open(in_path,encoding=p_encoding) as in_handler :
        for each_word in in_handler:
            each_word = each_word.strip()
            out_set.add(each_word) 
            

def read_dict():
    p_set = set()
    n_set = set()
    inroot = "/Users/zhangchunyu/Downloads/projects/feedback_analysis"
    tsinghua_name = "清华大学李军中文褒贬义词典"
    read_file(os.path.join(inroot,tsinghua_name,"tsinghua.positive.gb.txt"),p_set,"gbk")
    read_file(os.path.join(inroot,tsinghua_name,"tsinghua.negative.gb.txt"),n_set,"gbk")
    ntusd_name = "台湾大学情感词典"
    read_file(os.path.join(inroot,ntusd_name,"ntusd-positive.txt"),p_set)
    read_file(os.path.join(inroot,ntusd_name,"ntusd-negative.txt"),n_set)
    return p_set,n_set

def read_stop_words(path):
    stop_set = set()
    with open(path) as in_handler:
        for each_line in in_handler:
            each_line = each_line.strip()
            stop_set.add(each_line)
    return stop_set

def sen_preprocess(data,stop_words,out_path):
    word_dict = {}
    for each in data:
        word_list = jieba.cut(each[data_meta["content_teacher"]],cut_all=False)
        for each_word in word_list:
            if each_word in word_dict:
                word_dict[each_word] += 1
            else:
                word_dict[each_word] = 1
    return word_dict
    

['1声音小', '2上课期间对方环境噪声大', '3老师中间离开处理噪声', '4教学时长不足', '5不能很好调动小朋友积极性']


In [212]:
parent_in_root = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/"
parent_out = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/parent_feedback_step1_v2"
stop_word_path = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/cn_stop_word.txt"
#in_data = read_data(parent_in_root)
#p_set,n_set = read_dict()
stop_words = read_stop_words(stop_word_path)
preprocess_data(parent_in_root,parent_out,stop_words)


/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000004_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000000_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000003_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000006_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000002_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000005_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000001_0


In [202]:
def lda_topic(in_path):
    te = []
    with open(in_path) as in_handler:
        for each_line in in_handler:
            each_line = each_line.strip()
            line_arr = each_line.split()
            te.append(line_arr)
    dictionary = corpora.Dictionary(te)
    corpus = [dictionary.doc2bow(text) for text in te]
    lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=50)
    return lda

lad = lda_topic(parent_out)

In [216]:
def lda_tfidf_topic(in_path):
    te = []
    with open(in_path) as in_handler:
        for each_line in in_handler:
            each_line = each_line.strip()
            line_arr = each_line.split()
            te.append(line_arr)
    dictionary = corpora.Dictionary(te)
    corpus = [dictionary.doc2bow(text) for text in te]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lda = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=200)
    return lda

lda_tfidf = lda_topic(parent_out)

In [257]:
good_rate_words,bad_rate_words,good_df,bad_df = preprocess_data_v2(parent_in_root,stop_words)

/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000004_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000000_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000003_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000006_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000002_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000005_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_parent_feedback/000001_0


In [260]:
import math
def word_select(good_rate_words,bad_rate_words,good_df,bad_df):
    good_word_select = {}
    bad_word_select = {}
    for each_word in good_rate_words.keys():
        tf = good_rate_words[each_word]["tf"]
        in_bad_df = 1
        if each_word in bad_rate_words:
            in_bad_df = bad_rate_words[each_word]["df"]
        score = math.log(tf) * math.log(float(bad_df)/in_bad_df)
        good_word_select[each_word] = {"score":score,"tf":tf,"in_bad_df":in_bad_df}
        
    for each_word in bad_rate_words.keys():
        tf = bad_rate_words[each_word]["tf"]
        in_good_df = 1
        if each_word in good_rate_words:
            in_good_df = good_rate_words[each_word]["df"]
        score = 0
        if tf > 5:
            score = math.log(tf) * math.log(float(good_df)/in_good_df)
        bad_word_select[each_word] = {"score":score,"tf":tf,"in_good_df":in_good_df}
    good_sort = sorted(good_word_select.items(),key = lambda x:x[1]["score"],reverse=True)
    bad_sort = sorted(bad_word_select.items(),key = lambda x:x[1]["score"],reverse=True)
    return good_sort,bad_sort
    

In [261]:
print(good_df,bad_df)
good_sort,bad_sort = word_select(good_rate_words,bad_rate_words,good_df,bad_df)
for i in range(500):
    print(good_sort[i],bad_sort[i])

4886160 60624
('祝您', {'score': 110.37142068573813, 'tf': 22526, 'in_bad_df': 1}) ('最差', {'score': 64.3587827455686, 'tf': 685, 'in_bad_df': 256})
('holiday', {'score': 103.0219466792644, 'tf': 11557, 'in_bad_df': 1}) ('不负责任', {'score': 64.25658620017295, 'tf': 616, 'in_bad_df': 221})
('enjoys', {'score': 102.52393146083816, 'tf': 11046, 'in_bad_df': 1}) ('差评', {'score': 62.57804697754138, 'tf': 759, 'in_bad_df': 390})
('祝福', {'score': 102.2207061884113, 'tf': 10746, 'in_bad_df': 1}) ('无精打采', {'score': 61.45446549263253, 'tf': 759, 'in_bad_df': 462})
('课见', {'score': 100.91209187101731, 'tf': 9542, 'in_bad_df': 1}) ('应付', {'score': 58.38740218201011, 'tf': 799, 'in_bad_df': 785})
('🌹', {'score': 99.93519243864078, 'tf': 8732, 'in_bad_df': 1}) ('敷衍', {'score': 58.19857292796812, 'tf': 1180, 'in_bad_df': 1305})
('Looking', {'score': 99.21740069138303, 'tf': 8181, 'in_bad_df': 1}) ('无表情', {'score': 57.84530602065269, 'tf': 338, 'in_bad_df': 237})
('最爱', {'score': 99.1489252394632, 'tf': 22

In [139]:
black_list = ["","","","","",]
def get_word_dict(data):
    word_dict = {}
    for each in data:
        word_list = jieba.cut(each[data_meta["content_teacher"]],cut_all=False)
        for each_word in word_list:
            if each_word in word_dict:
                word_dict[each_word] += 1
            else:
                word_dict[each_word] = 1
    return word_dict

word_dict = get_word_dict(in_data)
sort_entry = sorted(word_dict.items(),key = lambda x : x[1], reverse=True)

In [140]:
def select_pn(data,p_dict,n_dict):
    p_ret = []
    n_ret = []
    print(len(p_dict),len(n_dict))
    for each in data:
        #print(each)
        if each[0] in p_dict:
            p_ret.append(each)
        if each[0] in n_dict:
            n_ret.append(each)
    return p_ret,n_ret

p_list,n_list = select_pn(sort_entry,p_set,n_set)

7408 10984


In [146]:
print(len(p_list),len(n_list))
for i in range(500):
    print(p_list[i])

3539 3403
('老师', 4731740)
('喜欢', 1430397)
('耐心', 762197)
('谢谢', 714909)
('希望', 422312)
('引导', 278450)
('还', 271907)
('鼓励', 260177)
('学习', 242574)
('感谢', 228154)
('不错', 224312)
('特别', 222397)
('认真', 207763)
('开心', 204341)
('期待', 197065)
('很棒', 166174)
('及时', 164013)
('满意', 126785)
('宝贝', 107879)
('热情', 107488)
('积极', 103242)
('很多', 89669)
('活跃', 82831)
('丰富', 82124)
('拓展', 78361)
('有趣', 77524)
('进步', 76155)
('帮助', 75618)
('活泼', 73398)
('努力', 69675)
('知道', 68047)
('愉快', 66624)
('指导', 60360)
('准备', 59336)
('负责', 58650)
('用心', 56542)
('高兴', 55845)
('完成', 54643)
('快乐', 52685)
('幽默', 52503)
('教导', 49110)
('兴趣', 47354)
('清晰', 44916)
('理解', 43764)
('超级', 40775)
('进行', 40301)
('温柔', 38887)
('细心', 38845)
('可爱', 38767)
('亲和力', 38623)
('积极性', 38572)
('亲切', 37804)
('宝宝', 37653)
('完整', 36093)
('注意', 35910)
('吸引', 35564)
('适合', 35483)
('清楚', 34998)
('详细', 34605)
('节奏', 34386)
('机会', 33723)
('掌握', 33285)
('发现', 31439)
('坚持', 31260)
('轻松', 31136)
('细致', 30451)
('效果', 30276)
('生动', 28520)
('适当', 28387)


In [160]:
data_teacher_meta={
    "teacher_comment_id" : 0,
    "ability_to_follow_instructions" : 1,
    "actively_interaction" : 2,
    "clear_pronunciation" : 3,
    "empty" : 4,
    "reading_skills" : 5,
    "repetition" : 6,
    "report_issues" : 7,
    "spelling_accuracy" : 8,
    "stars" : 9,
    "teacher_feedback" : 10,
    "tips_for_other_teachers" : 11,
    "performance" : 12,
    "current_performance" : 13,
    "course_type" : 14,
    "performance_adjust" : 15,
    "create_date_time" : 16,
    "online_class_id" : 17,
}

def read_hownet_file(in_path,out_set,encoding="utf-8"):
    print(in_path)
    with open(in_path,encoding=encoding) as in_handler:
        strip_n = 2
        i = 0
        for each_line in in_handler:
            if i < 2:
                i += 1
                continue                
            
            #print(each_line)
            out_set.add(each_line.strip())

def read_dict_en():
    p_set = set()
    n_set = set()
    en_in_root = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/知网Hownet情感词典"
    p_name_list = ["正面评价词语（英文）.txt","正面情感词语（英文）.txt"]
    n_name_list = ["负面评价词语（英文）.txt","负面情感词语（英文）.txt"]
    for each_name in p_name_list:
        read_hownet_file(os.path.join(en_in_root,each_name),p_set,"latin1")
        
    for each_name in n_name_list:
        read_hownet_file(os.path.join(en_in_root,each_name),n_set,"latin1")
    return p_set,n_set

tea_in_root_path = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_teacher_comment"
t_in_data = read_data(tea_in_root_path)
t_p_set,t_n_set = read_dict_en()

/Users/zhangchunyu/Downloads/projects/feedback_analysis/tmp_teacher_comment/000000_0
/Users/zhangchunyu/Downloads/projects/feedback_analysis/知网Hownet情感词典/正面评价词语（英文）.txt
/Users/zhangchunyu/Downloads/projects/feedback_analysis/知网Hownet情感词典/正面情感词语（英文）.txt
/Users/zhangchunyu/Downloads/projects/feedback_analysis/知网Hownet情感词典/负面评价词语（英文）.txt
/Users/zhangchunyu/Downloads/projects/feedback_analysis/知网Hownet情感词典/负面情感词语（英文）.txt


In [177]:
#import nltk.data
#nltk.download('punkt')
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
def get_word_dict(in_data):
    black_list = [",",".","!","?"]
    stop_words = set(stopwords.words('english'))
    stop_words.update(black_list)
    print(stop_words)
    ret = {}
    i = 0
    for each_data in in_data:
        #fields = each_line.split("\001")
        feedback = each_data[data_teacher_meta["teacher_feedback"]]
        words = word_tokenize(feedback)
        for each_word in words:
            each_word = each_word.lower()
            if not each_word in stop_words:
                if each_word in ret:
                    ret[each_word] += 1
                else:
                    ret[each_word] = 1
        
    return ret
en_word_dict = get_word_dict(t_in_data)

{'now', 'have', 'will', 'hers', 'where', 's', 'does', 'had', 'further', 'very', 'or', 'nor', 'same', 'no', 'which', 'him', 'ourselves', 'herself', 'we', 'as', 'to', 'should', 'over', 'is', 'on', 'itself', 'themselves', 'doing', 'because', 'in', 'few', 'until', 'below', 'myself', 'her', 'the', 'be', 'this', 'too', 'at', 'they', 'from', 'has', 'yourself', 'own', 'yourselves', 'most', '?', 'only', 'how', 'himself', 'are', 'here', 'been', 'into', 'do', 'were', 'again', 'there', ',', 'of', '.', 'about', 'that', 'against', 'after', 'down', 'did', 'up', 'i', 'me', 'while', 'before', 'when', 'during', 'out', 'don', 'he', 'above', 'these', 'was', 'their', 'ours', 'just', 'my', 'being', 'each', 'not', 't', 'having', 'so', 'then', 'its', 'theirs', 'some', 'can', 'a', 'them', 'but', 'both', 'such', 'off', 'more', 'and', 'your', 'what', 'if', 'between', 'than', 'all', 'it', 'those', 'she', 'am', 'an', 'who', 'through', 'once', 'you', 'under', 'other', 'with', '!', 'our', 'why', 'whom', 'by', 'for',

In [181]:
def p_n_select(data_dict,p_set,n_set):
    p_dict={}
    n_dict={}
    for each_key in data_dict.keys():
        if each_key in p_set:
            p_dict[each_key] = data_dict[each_key]
        if each_key in n_set:
            n_dict[each_key] = data_dict[each_key]
    p_sort = sorted(p_dict.items(),key = lambda x:x[1],reverse=True)
    n_sort = sorted(n_dict.items(),key = lambda x:x[1],reverse=True)
    return p_sort,n_sort

p_sort,n_sort = p_n_select(en_word_dict,t_p_set,t_n_set)

In [184]:
for i in range(50):
    print(p_sort[i])
print("################")
for i in range(200):
    print(n_sort[i])

('great', 1788554)
('well', 1052905)
('able', 930154)
('learned', 691597)
('good', 661268)
('thank', 618259)
('hope', 511776)
('excellent', 392168)
('new', 374785)
('like', 330407)
('wonderful', 299215)
('complete', 248741)
('nice', 218632)
('keep', 215183)
('happy', 199772)
('sound', 194573)
('correctly', 170507)
('really', 155694)
('meet', 152290)
('correct', 149818)
('thanks', 134485)
('big', 132097)
('love', 125185)
('long', 122782)
('smart', 121536)
('first', 110026)
('final', 95071)
('fantastic', 91871)
('quickly', 83764)
('easily', 78771)
('best', 71659)
('level', 66635)
('yes', 63925)
('even', 61551)
('due', 55539)
('proud', 54988)
('sweet', 54473)
('important', 51711)
('want', 51486)
('encourage', 51090)
('need', 51038)
('fluency', 48502)
('kind', 45280)
('focus', 43994)
('appreciate', 42360)
('honor', 41373)
('bright', 41144)
('attentive', 40432)
('back', 40188)
('needed', 39916)
################
('hard', 203790)
('used', 129529)
('small', 105991)
('question', 105847)
('fanta

In [311]:
def process_data_agg(data_agg,teacher_id):
    avg_rating = 100
    sort_list = sorted(data_agg, key = lambda x:x["scheduled_date_time"],reverse=True)
    min_len = min(len(sort_list),5)
    rating_sum = 10
    rating_list = []
    for i in range(len(sort_list)):
        rating = float(sort_list[i]["teacher_rating"])
        content = sort_list[i]["content_teacher"]
        if rating < 3.0:
           print(teacher_id,rating,content,sort_list[i]["scheduled_date_time"])
        #rating_sum += rating
        if content and len(content.strip())>0:            
            rating_list.append(rating)
            if rating < rating_sum:
                rating_sum=rating
    avg_rating = rating_sum/min_len
    return rating_sum,rating_list#avg_rating

def tea_rating_analysis(path):
    ret_dict = {}
    with open(path) as in_handler:
        i = 0
        for each in in_handler:
            if i == 0:
                i +=1
                continue
            index = each.find(",")
            teacher_id = each[:index]
            data_agg = each[index+1:]
            data_agg_r = data_agg.replace('""','"')
            data_agg_r = data_agg_r.lstrip('"')
            data_agg_r = data_agg_r.rstrip('"\n')
            #print(data_agg_r)
            data_obj = json.loads(data_agg_r)
            avg_rating,rating_list = process_data_agg(data_obj,teacher_id)
            #print(avg_rating)
            ret_dict[teacher_id] = {"score":avg_rating,"list":rating_list}
            #ret_list.append(data_obj)
    sort_list = sorted(ret_dict.items(),key = lambda x : x[1]["score"])
    #print(sort_list[0])
    return sort_list  

In [312]:
tea_agg_path = "/Users/zhangchunyu/Downloads/projects/feedback_analysis/teacher_rating_agg.csv"
agg_list = tea_rating_analysis(tea_agg_path)

1113458 1.0 因为看到了老师关注度特别高，就约了一节课，正好赶上是测评课，孩子有一些题做的不太正确，还有些时候孩子说什么，老师听不到，不知道是不是网络问题，但也不至于给出了40分，中教班主任在看了课程回放以后，也觉得不可思议，老师在整个课堂教学中一直在说GOOD，EXCELLENT，但测评时却给出这样的评价，让孩子不知道怎么回事。 2019-01-10 17:00:00
1169544 1.0 很一般的老师 讲课能把人讲的睡着了。关注量高不过是入职时间早罢了 2019-01-10 13:00:00
1217128 2.0 第一次上这个老师的课，但老师时间把握很不好，很快！大约15分钟就把课上完了，课程中 孩子说的很多有错的，也没纠正，反正就是很快的播放PPT！15分钟上完课，就给孩子看视频，看的时候也应该讲话啊 两个人默默的看，这是练啥英语？？ 以前也有老师很快上完的，也就提前5分钟，这次 我真是挺无语…… 2019-01-08 19:30:00
1245064 2.0 老师很疲劳 2019-01-12 18:00:00
1284164 1.0 老师一直在不停的喝水，对孩子没有引导，没有什么交流，很不满意。 2019-01-10 19:00:00
1524384 2.0 老师这节课打瞌睡了。。。 2019-01-09 19:30:00
1952834 2.0 第一次尝试这个老师的课，因为开始的时候孩子声音小了点，老师就不停的重复之前的问题。孩子对着ppt，老师没有什么表情的让选择对的词语，孩子不认识单词，就问我是什么意思，下课后，孩子说不喜欢上这个老师的课，因为他不会的不教他。老师在24分钟的时候就开始说再见，到25关闭了。 2019-01-13 09:30:00
2036712 1.0 没见过这么评价孩子的老师，你没有积极性怪罪到孩子身上，一直再说孩子不听讲，课都没有好好上，放心绝对不会再订你的了 2019-01-09 19:00:00
2085086 2.0  2019-01-09 20:00:00
2152900 1.0 不知道什么原因，每次我们上完课优先预约老师的课程，都不通过，请说明原因，谢谢，既然不通过，就不要开通优先预约 2019-01-13 08:30:00
2214298 2.0 老师讲课太快了，25分钟的课程，17分钟就讲完了，剩下8分

2443051 1.0 The lesson was boring and not interactive! The teacher’s instructions were unclear and we actually didn’t know what the teacher wanted us to do. The teacher just kept on talking without appreciating what my girl had talked and done. This was the worst lesson we ever had in vipkid... 2019-01-11 21:00:00
2499313 2.0 不能说老师在授课中有什么错误之处，只是与优秀的老师相比，还是存在很大差距的，活跃课堂气氛，激发孩子学习兴趣是一名优秀教师需要具备的基本教学能力。但是这名老师与vipkid宣传的美国外教老师热情洋溢，善于鼓励，真是相差甚远。孩子上完课对我说，我不想评价她如何，反正我以后不想再上她的课了。 2019-01-11 21:00:00
2519025 2.0 不能对的起老师这个称号，真的很不耐心，孩子电脑上出现了广告遮住了视频但是无法一下子关闭，又无法对老师用英语解释，就被老师用很恶劣的词粗暴对待，孩子很伤心 2019-01-12 18:00:00
2581285 1.0  2019-01-10 20:30:00
2904009 1.0  2019-01-12 20:00:00
2914305 1.0 不好，一点都不好，没精打采，还有2个星星的时候，直接就下课了，感觉没上完课，差评 2019-01-08 21:00:00
2943673 1.0  2019-01-09 18:00:00
3034689 1.0  2019-01-12 20:00:00
3438507 2.0  2019-01-08 19:30:00
3476181 1.0 老师很差，网络出现问题，技术人员跟她沟通让她回到前一页，让小朋友可以把歌曲唱完。她直接拒绝，最后三分钟问孩子跟课程毫无关系的问题消磨时间，比如：“这个笑脸图标什么颜色，这个哭脸图标什么颜色”等等。只能说见过不负责任的老师，但没见过这么不负责任的。投诉VIPJID客服，竟然也敷衍，拒绝补课，拒绝补课

22764445 2.0  2019-01-09 19:00:00
22979465 2.0  2019-01-14 19:00:00
23201645 1.0  2019-01-09 18:30:00
23271727 1.0  2019-01-12 20:30:00
24317185 2.0  2019-01-13 20:30:00
25130037 2.0  2019-01-08 18:30:00
25349909 2.0 估计老师没提前看一下课件，节奏掌握的很不好，只上了一半的内容。 2019-01-09 19:00:00
26827931 2.0 授课形式呆板，内容单一，很难调动孩子的积极性，上课节奏掌控不合理 2019-01-11 19:30:00
26827931 2.0 第一次在平台上给老师打低分，我不知道是老师准备不足还是第一次教孩子，第一过程中孩子名字都说错（两次），第二课件内容自己读错，that is 读成that was 让孩子莫名其妙，第三上课过程中多次冷场，我能听到很长时间的沉默，发生在课件和课件转换之间 2019-01-09 19:30:00
26828051 1.0 我精挑细选的老师，满怀期待，一堂课下来没有一点笑容，请问一下哪个小朋友愿意上这样老师的课呢？上课时间没到就撤了，最离奇的是课后评语，就给了孩子两句话，他阅读能力很好啥的…真是第一次有这样老师才给两句评语的。一节课费用那么高，真是不想说多了，不会再约 2019-01-11 17:30:00
26829727 1.0 不会引导 2019-01-11 21:00:00
27369383 1.0 老师情绪一点都不积极，而且网络质量超烂，一节课实际只上了半节 2019-01-11 10:00:00
28363733 1.0  2019-01-12 17:00:00


In [308]:
for i in range(200):
    print(agg_list[i])

('1113458', {'score': 1.0, 'list': [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]})
('1169544', {'score': 1.0, 'list': [5.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, 5.0, 5.0]})
('1284164', {'score': 1.0, 'list': [1.0]})
('2036712', {'score': 1.0, 'list': [5.0, 1.0, 5.0, 5.0]})
('2152900', {'score': 1.0, 'list': [1.0, 5.0, 5.0, 5.0, 5.0]})
('2256902', {'score': 1.0, 'list': [1.0, 5.0, 5.0]})
('2257188', {'score': 1.0, 'list': [5.0, 5.0, 3.0, 1.0, 3.0]})
('2298772', {'score': 1.0, 'list': [1.0, 5.0]})
('2370880', {'score': 1.0, 'list': [1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]})
('2738950', {'score': 1.0, 'list': [5.0, 5.0, 1.0]})
('3231446', {'score': 1.0, 'list': [3.0, 1.0]})
('3363344', {'score': 1.0, 'list': [1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]})
('3405478', {'score': 1.0, 'list': [1.0]})
('3463070', {'score': 1.0, 'list': [5.0, 1.0]})
('3466050', {'score': 1.0, 'list': [4.0, 5.0, 1.0]})
('3654010', {'score': 1.0, 'list': [1.0, 5.0, 5.0,